In [129]:
import pickle
with open('transcripts_metadata_records.pkl','rb') as infile:
    data = pickle.load(infile)

In [130]:
data

[{'vid_id': 'Q75JgLEXMsM',
  'title': 'LLM Zoomcamp 1.1 - Introduction to LLM and RAG',
  'timecode': '00:00',
  'text': "hi everyone Welcome to our course this is our first module for first unit so in this course the course is called llm Zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation I'll shortly talk about these variations what they mean um and what we exactly will do and I want to start first with explaining the problem we are going to use uh to solve throughout the course um so this will be our running problem and in our community in data do club we have multiple courses so this llm Zoom Camp is our fifth course and usually in our courses we have frequently asked questions so there are questions that uh there are no answers in the videos or answers are not uh easy to find and we have these documents I'll quickly open one of them and in these documents we have frequently asked qu

In [80]:
transcripts = []
for k, v in data.items():
    formatted = {}
    formatted['title'] = k
    formatted['transcript'] = v
    transcripts.append(formatted)

In [83]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
for d in transcripts:
    d['search_vector'] = model.encode(d['transcript'])

/home/gary/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-09-05 18:01:30.533262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 18:01:30.553312: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 18:01:30.553353: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-05 18:01:30.566747: I tensorflow/core/platform/cpu_feature_gua

In [86]:
# import pickle
# with open('full_transcripts_search.pkl','wb') as outfile:
#     pickle.dump(transcripts, outfile)

In [96]:
es = Elasticsearch("http://elasticsearch:9200")
if not es.ping():
    raise ValueError("Connection to Elasticsearch failed!")

ValueError: Connection to Elasticsearch failed!

In [98]:
import pickle
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

# Load the model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Connect to Elasticsearch
es = Elasticsearch("http://localhost:9200")
index_name = "full_transcripts"

# Define the index settings
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "transcript": {"type": "text"},
            "search_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

# Delete existing index if it exists, then create a new one
es.indices.delete(index=index_name, ignore_unavailable=True)
es.indices.create(index=index_name, body=index_settings)


for doc in data:
    try:
        es.index(index=index_name, document=doc)
    except Exception as e:
        print(f"Failed to index document {doc}: {e}")



In [17]:
recs = []
for k,v in data.items():
    recs.append({k:v})

In [20]:
import hashlib
def gen_ids(data):
    for k,v in data.items():
        new = {}
        new[k]=v
        hash_object = hashlib.md5(k.strip().encode())
        hash_hex = hash_object.hexdigest()
        new['id'] = hash_hex
    return new


In [21]:
id_recs = []
for i, r in enumerate(recs):
    id_recs.append(gen_ids(r))

In [55]:
len(id_recs)

48

In [114]:
data

[{'title': 'LLM Zoomcamp 1.1 - Introduction to LLM and RAG',
  'transcript': "hi everyone Welcome to our course this is our first module for first unit so in this course the course is called llm Zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation I'll shortly talk about these variations what they mean um and what we exactly will do and I want to start first with explaining the problem we are going to use uh to solve throughout the course um so this will be our running problem and in our community in data do club we have multiple courses so this llm Zoom Camp is our fifth course and usually in our courses we have frequently asked questions so there are questions that uh there are no answers in the videos or answers are not uh easy to find and we have these documents I'll quickly open one of them and in these documents we have frequently asked questions so the format is there is a section 

In [116]:
def make_prompt(records):
    return f"""
You are a system that generates potential questions based on the idea that there is an Online Course which teaches Retrieval Augmented Generation (RAG)
and using Large Language Models to improve the accuracy of using LLMs. Determining metrics for measuring performance of RAG and LLM answers. This course also covers
Elasticsearch and best methods for retrieval. The following is a dictionary which has a video with the title as a key the full transcript for each video as 
the value for the transcript key. For each record, please generate three potential questions a student might ask where the record in question would be the best record  
Please return your responses in the form of a list of dictionaries. So each title should have three records returned all should have the same title and transcript, but a third 
field called 'student_question' where a unique question is asked where that video out of all the videos is the most suitable choice. Again, do this for ALL records. So that if 
there are 5 dictionaries in the input list below there should be 15 dictionaries returned in the list. Please do not include the ```json or any other text. Just the list of 
dictionaries with id, the video title with the transcript as the value and the student question you generate.


INPUT RECORDS:
{records}
"""

In [124]:
prompt = make_prompt(data)

In [125]:
from openai import OpenAI
client = OpenAI()

In [126]:
def add_keywords(prompt):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"{prompt}"},
            # {
            #     "role": "user",
            #     "content": f"{data}"
            # }
        ]
    )

    return completion.choices[0].message

In [127]:
ground_truth = add_keywords(prompt)

In [123]:
prompt

"\nYou are a system that generates potential questions based on the idea that there is an Online Course which teaches Retrieval Augmented Generation (RAG)\nand using Large Language Models to improve the accuracy of using LLMs. Determining metrics for measuring performance of RAG and LLM answers. This course also covers\nElasticsearch and best methods for retrieval. The following is a dictionary which has a video with the title as a key the full transcript for each video as \nthe value for the transcript key. For each record, please generate three potential questions a student might ask where the record in question would be the best record  \nPlease return your responses in the form of a list of dictionaries. So each title should have three records returned all should have the same title and transcript, but a third \nfield called 'student_question' where a unique question is asked where that video out of all the videos is the most suitable choice. Again, do this for ALL records. So that

In [121]:
ground_truth.content.replace('\n','')

'[]'

In [70]:
cleaning = [x.replace('\n','') for x in ground_truth.content]

AttributeError: 'dict' object has no attribute 'replace'

In [75]:
ast.literal_eval(ground_truth.content.replace('\n',''))

[{'id': 'b80bb7740288fda1f201890375a60c8f',
  'LLM Zoomcamp 1.2 - Configuring Your Environment': "hi everyone in this video I want to show you how to configure your environment in this particular video I want to use GitHub codes spaces to show how to configure it but you don't have to use it you can just follow along the video and do everything you need on your local machine I will not show how to install Docker you will need to do it yourself because Cod spaces already has Docker and we will focus on installing uh the libraries we need for this particular model or you can like you again as I said you don't have to use cod spaces it's a very convenient tool but you can also for this model you can use G collab Google collab you can use C Cloud uh you can use any other notebook provider like Sage maker or you can just run things locally for the uh second module we will need a GPU right now we don't need your GPU for the second module we will so if you have an access to a machine with GPU

In [62]:
import json

# Assuming this is your ChatCompletionMessage object
api_response = ground_truth.content

# Clean up the content by removing code block markers and extra text
json_string = api_response.strip().strip('```json').strip('```').strip()

# Verify the cleaned JSON string
print(f"Cleaned JSON string: {json_string}")

# Now, parse the JSON string into a Python dictionary
try:
    data = json.loads(json_string)
    # Iterate over the dictionary
    for key, value in data.items():
        print(f"Key: {key}")
        print(f"Value: {value}")
except json.JSONDecodeError as e:
    print(f"Failed to decode JSON: {e}")


Cleaned JSON string: [
    {
        "id": "b80bb7740288fda1f201890375a60c8f",
        "LLM Zoomcamp 1.2 - Configuring Your Environment": "hi everyone in this video I want to show you how to configure your environment in this particular video I want to use GitHub codes spaces to show how to configure it but you don't have to use it you can just follow along the video and do everything you need on your local machine I will not show how to install Docker you will need to do it yourself because Cod spaces already has Docker and we will focus on installing uh the libraries we need for this particular model or you can like you again as I said you don't have to use cod spaces it's a very convenient tool but you can also for this model you can use G collab Google collab you can use C Cloud uh you can use any other notebook provider like Sage maker or you can just run things locally for the uh second module we will need a GPU right now we don't need your GPU for the second module we will so if

AttributeError: 'list' object has no attribute 'items'

In [53]:
data["b57447068c829d5cc1443855322d7727"]["student_question"]

KeyError: 'b57447068c829d5cc1443855322d7727'

In [46]:
import pickle

# Assuming `data` is the dictionary you want to save
with open('ground_truth_full_trans.pkl', 'wb') as pickle_file:
    pickle.dump(data, pickle_file)

In [47]:
inspect = []
for i in data:
    inspect.append(i)

In [48]:
inspect[0]

'LLM Zoomcamp 1.1 - Introduction to LLM and RAG'

In [50]:
for key, value in data.items():
    print(f"Key: {key}")
    print(f"Value: {value}")

Key: LLM Zoomcamp 1.1 - Introduction to LLM and RAG
Value: hi everyone Welcome to our course this is our first module for first unit so in this course the course is called llm Zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation I'll shortly talk about these variations what they mean um and what we exactly will do and I want to start first with explaining the problem we are going to use uh to solve throughout the course um so this will be our running problem and in our community in data do club we have multiple courses so this llm Zoom Camp is our fifth course and usually in our courses we have frequently asked questions so there are questions that uh there are no answers in the videos or answers are not uh easy to find and we have these documents I'll quickly open one of them and in these documents we have frequently asked questions so the format is there is a section then there's a ques

In [54]:
with open("ground_truth_full_trans_raw.txt",'w') as outfile:
    outfile.writelines(ground_truth.content)